<a href="https://colab.research.google.com/github/milver/Experiments/blob/main/Sentinel_LULC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experimenting with Sentinel Land Use Land Cover classification

In [15]:
#%pip install rasterio
import ee
import geemap
import rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 45.8 MB/s eta 0:00:00


## Get Bounding Box
Setting up Earth Engine credentials

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-my-milver')


In [16]:
#!pip install geemap --upgrade
Map = geemap.Map()
Map.add_basemap("Esri.NatGeoWorldMap")
Map.set_center(-77.5107765197754, 39.01424883668118, 13)
Map

Map(center=[39.01424883668118, -77.5107765197754], controls=(WidgetControl(options=['position', 'transparent_b…

In [17]:

Map.bounds

((39.00491168506387, -77.76689529418947),
 (39.08490440902499, -77.45927810668947))

### Set the region of interest by simply drawing a polygon on the map

In [18]:
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(Map.west,  Map.south, Map.east, Map.north)

Map.centerObject(region)

In [19]:
# bbox1 = [m.west, m.south, m.east, m.north]
# print("{}: {}".format(m.center, m.zoom))
print("{}: {}".format(Map.center, Map.zoom))

[39.044908047815255, -77.61308670043945]: 13.0


## Extracting Sentinel-2
Construct a collection of corresponding Dynamic World and Sentinel-2 for inspection. Filter the DW and S2 collections by region and date.

### Start the filtering

In [20]:
START = ee.Date('2021-04-02')
END = ee.Date('2024-05-10')  #START.advance(1, 'day')

# col_filter = ee.Filter.And(
#     ee.Filter.bounds(ee.Geometry.Point(-77.51601219177248, 39.01918369029137)),
#     ee.Filter.date(START, END),
# )

col_filter = ee.Filter.And(ee.Filter.bounds(region), ee.Filter.date(START, END),)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

### Construct a collection of corresponding Dynamic World and  Sentinel-2 for inspection.
Filter the DW and S2 collections by region and date.

In [21]:
# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)



### Clip Rasters by Region and display

In [26]:
# Display the Dynamic World visualization with the source Sentinel-2 image.
import geemap
m = geemap.Map()

mask = region
m.centerObject(region, int(Map.zoom))

m.add_layer(
    s2_image.clip(region),
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade.clip(region),
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)

m.add_legend(keys=CLASS_NAMES, colors=VIS_PALETTE, position="bottomleft")
m

Map(center=[39.04493416667043, -77.61308670044056], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
dw_image